In [1]:
import numpy as np
import xarray as xr
import pandas as pd
#from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Activation, MaxPool2D
#from tensorflow.keras.models import Model, save_model
import matplotlib.pyplot as plt
import FourierOpticsLib as FO
#from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

In [2]:
ds_path = "/scr/sci/mhayman/holodec/holodec-ml-data/"
ds_name = "synthethic_holograms_v0.nc"
ds = xr.open_dataset(ds_path+ds_name)
ds_base = ds_name.replace(".nc","")

In [3]:
# define the functions to be applied to the fourier transformed data
ft_func = {'real':np.real,'imag':np.imag,'amplitude':np.abs}
ft_scale = {'real':255,'imag':255,'amplitude':255}  # rescaling factors
encoded_dtype = "int16"

In [4]:
# initialize the image Fourier Transform channels
image_ft = {}
for func in ft_func.keys():
    image_ft[func] = xr.DataArray(np.zeros(ds['image'].shape,dtype='float32'),
                        dims=['ysize','xsize','hologram_number'])

In [5]:
# create a variable with the amplitude of the Fourier Transform of each image
for im in range(ds['image'].shape[2]):
    image_ft0 = FO.OpticsFFT(ds['image'][:,:,im]-np.mean(ds['image'][:,:,im]))
    for func in ft_func.keys():
        image_ft[func][:,:,im] = ft_func[func](image_ft0) / ft_scale[func]

In [6]:
image_ft = xr.concat(list(image_ft.values()),pd.Index(ft_func.keys(),name='channels'))

In [7]:
ds['image']

<xarray.DataArray 'image' (ysize: 400, xsize: 600, hologram_number: 10000)>
[2400000000 values with dtype=uint8]
Dimensions without coordinates: ysize, xsize, hologram_number
Attributes:
    longname:  Hologram image

In [8]:
ds_ft = ds.assign(image_ft=image_ft)

In [9]:
encode_defs = {'image_ft':{'dtype':encoded_dtype,
                           'scale_factor':(image_ft.max()-image_ft.min()).values/(np.iinfo(encoded_dtype).max-np.iinfo(encoded_dtype).min),
                           'add_offset':image_ft.mean().values}}

In [10]:
encode_defs

{'image_ft': {'dtype': 'int16',
  'scale_factor': 3.053771919249906e-05,
  'add_offset': array(0.00174452, dtype=float32)}}

In [11]:
ds_ft

<xarray.Dataset>
Dimensions:   (channels: 3, hologram_number: 10000, particle: 10000, xsize: 600, ysize: 400)
Coordinates:
  * channels  (channels) object 'real' 'imag' 'amplitude'
Dimensions without coordinates: hologram_number, particle, xsize, ysize
Data variables:
    hid       (particle) float32 ...
    d         (particle) float32 ...
    x         (particle) float32 ...
    y         (particle) float32 ...
    z         (particle) float32 ...
    image     (ysize, xsize, hologram_number) uint8 ...
    image_ft  (channels, ysize, xsize, hologram_number) float32 0.0028177137 ... 0.0031704279
Attributes:
    DateCreated:           28-Oct-2019
    Nx:                    600.0
    Ny:                    400.0
    dx:                    2.96e-06
    dy:                    2.96e-06
    lambda:                3.55e-07
    Dpmin:                 2e-05
    Dpmax:                 7e-05
    zMin:                  0.014
    zMax:                  0.158
    NParticles:            1.0
    Nbit

In [12]:
ds_ft.to_netcdf(ds_path+ds_base+"_ft_ac_"+"_".join(ft_func.keys())+".nc",encoding=encode_defs)

In [14]:
ds.close()
ds_ft.close()